# Use BigQuery DataFrames to run Anthropic LLM at scale

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/remote_functions/remote_function_vertex_claude_model.ipynb">
      <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/Google_Colaboratory_SVG_Logo.svg/1600px-Google_Colaboratory_SVG_Logo.svg.png?20221103151432" alt="Colab logo" width="65"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/remote_functions/remote_function_vertex_claude_model.ipynb">
      <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Octicons-mark-github.svg/1200px-Octicons-mark-github.svg.png?20180806170715" alt="GitHub logo" width="35">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/remote_functions/remote_function_vertex_claude_model.ipynb">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35">
      Open in BQ Studio
    </a>
  </td>
</table>

In [ ]:
import sys

# Python 3.13 is not yet a supported runtime for remote functions.
# See: https://cloud.google.com/functions/docs/runtime-support#python for the supported runtimes.
if sys.version_info >= (3, 13, 0):
    sys.exit(0)

# Overview

Anthropic Claude models are available as APIs on Vertex AI ([docs](https://cloud.google.com/vertex-ai/generative-ai/docs/partner-models/use-claude)).

To run the Claude models at large scale data we can utilze the BigQuery
DataFrames remote functions ([docs](https://cloud.google.com/bigquery/docs/use-bigquery-dataframes#remote-functions)).
BigQuery DataFrames provides a simple pythonic interface `remote_function` to
deploy the user code as a BigQuery remote function and then invoke it at scale
by utilizing the parallel distributed computing architecture of BigQuery and
Google Cloud Function.

In this notebook we showcase one such example. For the demonstration purpose we
use a small amount of data, but the example generalizes for large data. Check out
various IO APIs provided by BigQuery DataFrames [here](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.pandas#bigframes_pandas_read_gbq)
to see how you could create a DataFrame from your Big Data sitting in a BigQuery
table or GCS bucket.

# Set Up

## Set up a claude model in Vertex

https://cloud.google.com/vertex-ai/generative-ai/docs/partner-models/use-claude#before_you_begin

## Install Anthropic with Vertex if needed

Uncomment the following cell and run the cell to install anthropic python
package with vertex extension if you don't already have it.

In [1]:
# !pip install anthropic[vertex] --quiet

## Define project and location for GCP integration

In [2]:
PROJECT = "bigframes-dev" # replace with your project
LOCATION = "us-east5"

# Initialize BigQuery DataFrames dataframe

BigQuery DataFrames is a set of open source Python libraries that let you take
advantage of BigQuery data processing by using familiar Python APIs.
See for more details https://cloud.google.com/bigquery/docs/bigquery-dataframes-introduction.

In [3]:
# Import BigQuery DataFrames pandas module and initialize it with your project
# and location

import bigframes.pandas as bpd
bpd.options.bigquery.project = PROJECT
bpd.options.bigquery.location = LOCATION

Let's use a DataFrame with small amount of inline data for demo purpose.
You could create a DataFrame from your own data. See APIs like `read_gbq`,
`read_csv`, `read_json` etc. at https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.pandas.

In [4]:
df = bpd.DataFrame({"questions": [
   "What is the capital of France?",
   "Explain the concept of photosynthesis in simple terms.",
   "Write a haiku about artificial intelligence."
 ]})
df

,questions
0,What is the capital of France?
1,Explain the concept of photosynthesis in simpl...
2,Write a haiku about artificial intelligence.


# Use BigQuery DataFrames `remote_function`

Let's create a remote function from a custom python function that takes a prompt
and returns the output of the claude LLM running in Vertex. We will be using
`max_batching_rows=1` to control parallelization. This ensures that a single
prompt is processed per batch in the underlying cloud function so that the batch
processing does not time out. An ideal value for `max_batching_rows` depends on
the complexity of the prompts in the real use case and should be discovered
through offline experimentation. Check out the API for other ways to control
parallelization https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.pandas#bigframes_pandas_remote_function.

In [6]:
@bpd.remote_function(packages=["anthropic[vertex]", "google-auth[requests]"],
                     max_batching_rows=1, 
                     bigquery_connection="bigframes-dev.us-east5.bigframes-rf-conn", # replace with your connection
                     cloud_function_service_account="default",
)
def anthropic_transformer(message: str) -> str:
  from anthropic import AnthropicVertex
  client = AnthropicVertex(region=LOCATION, project_id=PROJECT)

  message = client.messages.create(
              max_tokens=1024,
              messages=[
                  {
                      "role": "user",
                      "content": message,
                  }
              ],
              model="claude-3-5-sonnet@20240620",
          )
  content_text = message.content[0].text if message.content else ""
  return content_text

In [7]:
# Print the BigQuery remote function created
anthropic_transformer.bigframes_remote_function

'bigframes-dev._b52b272a35b88e236e1f96fbe3f560c83a8fee85.bigframes_session265649_de1176dd4c57f40ba959503af3981682'

In [8]:
# Print the cloud function created
anthropic_transformer.bigframes_cloud_function

'projects/bigframes-dev/locations/us-east5/functions/bigframes-session265649-de1176dd4c57f40ba959503af3981682'

In [9]:
# Apply the remote function on the user data
df["answers"] = df["questions"].apply(anthropic_transformer)
df

,questions,answers
0,What is the capital of France?,The capital of France is Paris.
1,Explain the concept of photosynthesis in simpl...,Photosynthesis is the process plants use to ma...
2,Write a haiku about artificial intelligence.,Here's a haiku about artificial intelligence: ...


# Clean Up

In [ ]:
bpd.close_session()